In [1]:
import gdown
gdown.download(id = '18uC7WTuEXKJDDxbj-Jq6EjzpFrgE7IAd', output = 'dogs-vs-cats.zip')

Downloading...
From: https://drive.google.com/uc?id=18uC7WTuEXKJDDxbj-Jq6EjzpFrgE7IAd
To: /content/dogs-vs-cats.zip
100%|██████████| 852M/852M [00:03<00:00, 239MB/s]


'dogs-vs-cats.zip'

In [2]:
!unzip -qq dogs-vs-cats.zip

In [3]:
!unzip -qq train.zip

In [4]:
import os, shutil, pathlib

In [5]:
og_dir = pathlib.Path("train")
new_dir = pathlib.Path("dogs_cats_small")
def make_subset(sub_n, st_idx, end_idx):
    for c in ("cat", "dog"):
        dir = new_dir /sub_n/c
        os.makedirs(dir)
        f_ns = [f'{c}.{i}.jpg' for i in range(st_idx, end_idx)]
        for f_n in f_ns:
            shutil.copyfile(src = og_dir /f_n, dst = dir / f_n)
make_subset("train_data", 0, 1000)
make_subset("val_data", 1000, 1500)
make_subset("test_data", 1500, 2500)


In [6]:
from keras.utils import image_dataset_from_directory
train_dataset = image_dataset_from_directory(new_dir/'train_data', batch_size = 32, image_size = (180, 180))
test_dataset = image_dataset_from_directory(new_dir/'test_data', batch_size = 32, image_size = (180, 180))
val_dataset = image_dataset_from_directory(new_dir/'val_data', batch_size = 32, image_size = (180, 180))

Found 2000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [8]:
import numpy as np
import tensorflow as tf
r_n = np.random.normal(size = (1000, 16))
dataset = tf.data.Dataset.from_tensor_slices(r_n)

In [11]:
for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break


(16,)
(16,)
(16,)


In [12]:
bc_dataset = dataset.batch(32)
for i, element in enumerate(bc_dataset):
    print(element.shape)
    if i >= 0:
        break

(32, 16)


In [13]:
reshape_dataset = dataset.map(lambda x:tf.reshape(x, (4, 4)))
for i, element in enumerate(reshape_dataset):
    print(element.shape)
    if i >= 2:
        break

(4, 4)
(4, 4)
(4, 4)


In [14]:
for data, label in train_dataset:
    print(data.shape)
    print(label.shape)
    break

(32, 180, 180, 3)
(32,)


In [16]:
from keras.layers.pooling.max_pooling2d import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Rescaling, Flatten, MaxPooling2D
m = Sequential()
m.add(Rescaling(1./255, input_shape = (180, 180, 3)))
m.add(Conv2D(32, 3, activation = "relu"))
m.add(MaxPooling2D(2))
m.add(Conv2D(64, 3, activation = "relu"))
m.add(MaxPooling2D(2))
m.add(Conv2D(128, 3, activation = "relu"))
m.add(MaxPooling2D(2))
m.add(Conv2D(256, 3, activation = "relu"))
m.add(MaxPooling2D(2))
m.add(Flatten())
m.add(Dense(1, activation = 'sigmoid'))
m.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 41, 128)       7

In [17]:
m.compile(loss = 'binary_crossentropy', metrics = 'acc')

In [ ]:
from keras.callbacks import ModelCheckpoint
ModelCheckpoint()

In [18]:
m.fit(train_dataset)

63/63 [==============================] - 183s 3s/step - loss: 0.7055 - acc: 0.5290
